Create several of your own examples in json format for recognizing a new type of entity (choose your own). Create a program that adds these examples to an existing spaCy model, training the model.

In [ ]:
import random
import spacy
from spacy.training import Example
import json

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
ner = nlp.get_pipe("ner")
ner.add_label("FLIGHT_NUMBER")

In [ ]:
with open("fly.json", "r") as f:
    train_data = json.load(f)

In [ ]:
examples = []
for item in train_data:
    text = item["text"]
    entities = []
    for entity in item["entities"]:
        entities.append((entity["start"], entity["end"], "FLIGHT_NUMBER"))
    examples.append(Example.from_dict(nlp.make_doc(text), {"entities": entities}))

In [ ]:
epochs = 20
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for i in range(epochs):
        random.shuffle(examples)
        for example in examples:
            nlp.update([example], sgd=optimizer)

In [ ]:
nlp.to_disk("trained_flight_model")

In [9]:
test_texts = [
    "When should I board flight LX321?",
    "Has IB2345 been cancelled?",
    "When should I board flight LX321?",
    "Has IB2345 been cancelled?",
]
for text in test_texts:
    doc = nlp(text)
    print(f"Text: {text}")
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    print()

Text: When should I board flight LX321?
Entities: [('LX321', 'FLIGHT_NUMBER')]

Text: Has IB2345 been cancelled?
Entities: [('IB2345', 'FLIGHT_NUMBER')]

Text: When should I board flight LX321?
Entities: [('LX321', 'FLIGHT_NUMBER')]

Text: Has IB2345 been cancelled?
Entities: [('IB2345', 'FLIGHT_NUMBER')]

